# 🔍 التكليف الرابع: تحليل الأخطاء (Debug Analysis)

## 🎯 الهدف
تحليل السبب الذي يجعل النموذج يصنف جميع التقييمات كسلبية، وتقديم حلول للإصلاح.

---

## 📋 المشكلة

عند تدريب نموذج تصنيف المشاعر، قد نواجه مشكلة حيث يقوم النموذج بتصنيف **جميع التقييمات كسلبية** بغض النظر عن المحتوى الفعلي.

هذا السلوك يشير إلى وجود مشكلة في:
- البيانات
- المعالجة
- النموذج
- أو التقييم

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

print("✅ تم تحميل المكتبات")

✅ تم تحميل المكتبات


---

## 1️⃣ الأسباب المحتملة للمشكلة

### 🔴 السبب الأول: انقلاب التوسيم (Label Flipping)

**المشكلة:** تم تبديل التصنيفات بالخطأ (الإيجابي أصبح سلبي والعكس)

**كيفية الكشف:**

In [2]:
# محاكاة مشكلة انقلاب التوسيم
sample_data = {
    'text': [
        'الخدمة ممتازة جداً',  # يجب أن يكون إيجابي
        'أسوأ تجربة في حياتي', # يجب أن يكون سلبي
        'المنتج رائع أنصح به',  # يجب أن يكون إيجابي
        'سيء جداً لا أنصح',     # يجب أن يكون سلبي
    ],
    'label_wrong': ['negative', 'positive', 'negative', 'positive'],  # خطأ!
    'label_correct': ['positive', 'negative', 'positive', 'negative']  # صحيح
}

df_sample = pd.DataFrame(sample_data)
print("📋 مثال على انقلاب التوسيم:")
df_sample

📋 مثال على انقلاب التوسيم:


,text,label_wrong,label_correct
0,الخدمة ممتازة جداً,negative,positive
1,أسوأ تجربة في حياتي,positive,negative
2,المنتج رائع أنصح به,negative,positive
3,سيء جداً لا أنصح,positive,negative


In [3]:
# كيفية التحقق: مراجعة عينة عشوائية يدوياً
def verify_labels(df, text_col='text', label_col='label', sample_size=10):
    """
    التحقق من صحة التوسيم بمراجعة عينة عشوائية
    """
    sample = df.sample(min(sample_size, len(df)))
    print("🔍 عينة للمراجعة اليدوية:")
    print("=" * 60)
    for idx, row in sample.iterrows():
        print(f"\nالنص: {row[text_col]}")
        print(f"التصنيف الحالي: {row[label_col]}")
        print("-" * 40)

# verify_labels(df, 'text', 'label')

### 🔴 السبب الثاني: عدم توازن الفئات (Class Imbalance)

**المشكلة:** عدد التقييمات السلبية أكبر بكثير من الإيجابية (أو العكس)

**كيفية الكشف:**

In [4]:
# محاكاة عدم توازن الفئات
def check_class_balance(df, label_col='label'):
    """
    فحص توازن الفئات
    """
    counts = df[label_col].value_counts()
    total = len(df)
    
    print("📊 توزيع الفئات:")
    print("-" * 40)
    for label, count in counts.items():
        percentage = (count / total) * 100
        print(f"{label}: {count} ({percentage:.1f}%)")
    
    # تحديد ما إذا كان هناك عدم توازن
    ratio = counts.max() / counts.min()
    if ratio > 2:
        print(f"\n⚠️ تحذير: نسبة عدم التوازن = {ratio:.2f}")
        print("   يُنصح باستخدام تقنيات موازنة البيانات")
    else:
        print(f"\n✅ البيانات متوازنة نسبياً (النسبة = {ratio:.2f})")
    
    return counts

# مثال على بيانات غير متوازنة
imbalanced_labels = ['negative'] * 900 + ['positive'] * 100
df_imbalanced = pd.DataFrame({'label': imbalanced_labels})
check_class_balance(df_imbalanced)

📊 توزيع الفئات:
----------------------------------------
negative: 900 (90.0%)
positive: 100 (10.0%)

⚠️ تحذير: نسبة عدم التوازن = 9.00
   يُنصح باستخدام تقنيات موازنة البيانات


label
negative    900
positive    100
Name: count, dtype: int64

In [5]:
# الحل: استخدام class_weight
from sklearn.utils.class_weight import compute_class_weight

def get_balanced_weights(y):
    """
    حساب الأوزان المتوازنة للفئات
    """
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    return dict(zip(classes, weights))

# مثال
sample_y = [0] * 900 + [1] * 100
weights = get_balanced_weights(sample_y)
print("⚖️ الأوزان المحسوبة:")
print(f"   الفئة 0 (سلبي): {weights[0]:.4f}")
print(f"   الفئة 1 (إيجابي): {weights[1]:.4f}")

⚖️ الأوزان المحسوبة:
   الفئة 0 (سلبي): 0.5556
   الفئة 1 (إيجابي): 5.0000


### 🔴 السبب الثالث: مشكلة في المعالجة (Preprocessing Issue)

**المشكلة:** خطوات التنظيف أزالت معلومات مهمة للتصنيف

In [6]:
# فحص تأثير المعالجة على البيانات
def analyze_preprocessing_impact(original, cleaned):
    """
    تحليل تأثير المعالجة على النصوص
    """
    original_lengths = [len(t.split()) for t in original]
    cleaned_lengths = [len(t.split()) for t in cleaned]
    
    avg_original = np.mean(original_lengths)
    avg_cleaned = np.mean(cleaned_lengths)
    reduction = ((avg_original - avg_cleaned) / avg_original) * 100
    
    print("📊 تحليل تأثير المعالجة:")
    print("-" * 40)
    print(f"متوسط طول النص الأصلي: {avg_original:.1f} كلمة")
    print(f"متوسط طول النص المنظف: {avg_cleaned:.1f} كلمة")
    print(f"نسبة التقليص: {reduction:.1f}%")
    
    if reduction > 50:
        print("\n⚠️ تحذير: تقليص كبير في النصوص!")
        print("   قد تكون المعالجة أزالت معلومات مهمة")
    
    # فحص النصوص الفارغة
    empty_count = sum(1 for t in cleaned if len(t.strip()) == 0)
    if empty_count > 0:
        print(f"\n⚠️ تحذير: {empty_count} نص أصبح فارغاً بعد المعالجة!")

### 🔴 السبب الرابع: خلل في العتبة (Threshold Issue)

**المشكلة:** عتبة التصنيف (0.5) غير مناسبة للبيانات

In [7]:
# تحليل توزيع الاحتمالات
def analyze_probability_distribution(model, X_test_vec, y_test):
    """
    تحليل توزيع احتمالات التنبؤ
    """
    if hasattr(model, 'predict_proba'):
        probs = model.predict_proba(X_test_vec)[:, 1]  # احتمال الفئة الإيجابية
        
        plt.figure(figsize=(10, 5))
        
        # توزيع الاحتمالات
        plt.subplot(1, 2, 1)
        plt.hist(probs, bins=30, edgecolor='black', alpha=0.7)
        plt.axvline(x=0.5, color='red', linestyle='--', label='عتبة 0.5')
        plt.xlabel('احتمال التصنيف الإيجابي')
        plt.ylabel('العدد')
        plt.title('توزيع الاحتمالات')
        plt.legend()
        
        # منحنى ROC
        from sklearn.metrics import roc_curve, auc
        fpr, tpr, thresholds = roc_curve(y_test, probs)
        roc_auc = auc(fpr, tpr)
        
        plt.subplot(1, 2, 2)
        plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('منحنى ROC')
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
        # اقتراح عتبة مثالية
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        print(f"\n💡 العتبة المثالية المقترحة: {optimal_threshold:.3f}")
        
        return probs, optimal_threshold
    else:
        print("⚠️ النموذج لا يدعم predict_proba")
        return None, None

---

## 2️⃣ أدوات التشخيص الشاملة

In [8]:
def full_debug_analysis(df, text_col, label_col, cleaned_col=None):
    """
    تحليل شامل للأخطاء
    """
    print("="*60)
    print("🔍 تحليل الأخطاء الشامل")
    print("="*60)
    
    # 1. فحص توازن الفئات
    print("\n1️⃣ فحص توازن الفئات:")
    check_class_balance(df, label_col)
    
    # 2. فحص النصوص الفارغة
    print("\n2️⃣ فحص النصوص الفارغة:")
    empty_texts = df[df[text_col].str.strip() == ''].shape[0]
    null_texts = df[text_col].isna().sum()
    print(f"   - نصوص فارغة: {empty_texts}")
    print(f"   - قيم null: {null_texts}")
    
    # 3. فحص طول النصوص
    print("\n3️⃣ إحصائيات طول النصوص:")
    text_lengths = df[text_col].str.len()
    print(f"   - الحد الأدنى: {text_lengths.min()}")
    print(f"   - الحد الأقصى: {text_lengths.max()}")
    print(f"   - المتوسط: {text_lengths.mean():.1f}")
    
    # 4. فحص التكرار
    print("\n4️⃣ فحص التكرار:")
    duplicates = df[text_col].duplicated().sum()
    print(f"   - نصوص مكررة: {duplicates} ({(duplicates/len(df))*100:.1f}%)")
    
    # 5. عينة من كل فئة
    print("\n5️⃣ عينة من كل فئة للمراجعة:")
    for label in df[label_col].unique():
        print(f"\n   [{label}]:")
        samples = df[df[label_col] == label][text_col].head(3)
        for s in samples:
            print(f"   • {s[:50]}..." if len(s) > 50 else f"   • {s}")
    
    print("\n" + "="*60)

# يمكن استخدامها هكذا:
# full_debug_analysis(df, 'text', 'label')

---

## 3️⃣ الحلول المقترحة

In [9]:
solutions = """
# 📋 تقرير تحليل الأخطاء والحلول

## المشكلة
النموذج يصنف جميع التقييمات كسلبية.

---

## الأسباب المحتملة والحلول

### 1. انقلاب التوسيم (Label Flipping)
**السبب:** تم تبديل التصنيفات بالخطأ
**الحل:**
```python
# عكس التصنيفات
df['label'] = df['label'].map({'positive': 'negative', 'negative': 'positive'})
```

---

### 2. عدم توازن الفئات (Class Imbalance)
**السبب:** فئة أكبر بكثير من الأخرى
**الحلول:**
```python
# الحل 1: استخدام class_weight
model = LogisticRegression(class_weight='balanced')

# الحل 2: Over-sampling للفئة الأقل
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# الحل 3: Under-sampling للفئة الأكبر
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)
```

---

### 3. مشكلة في المعالجة (Preprocessing)
**السبب:** التنظيف أزال كلمات مهمة للتصنيف
**الحلول:**
- مراجعة خطوات التنظيف
- الحفاظ على negation words (لا، ما، لن)
- تقليل التطبيع المفرط

---

### 4. مشكلة العتبة (Threshold)
**السبب:** عتبة 0.5 غير مناسبة
**الحل:**
```python
# استخدام عتبة مخصصة
probs = model.predict_proba(X_test)[:, 1]
optimal_threshold = 0.3  # أو قيمة محسوبة
predictions = (probs >= optimal_threshold).astype(int)
```

---

## التوصيات

1. ✅ مراجعة عينة عشوائية من البيانات يدوياً
2. ✅ التحقق من توازن الفئات
3. ✅ استخدام class_weight='balanced'
4. ✅ تحليل Confusion Matrix بعناية
5. ✅ تجربة عتبات مختلفة
"""

print(solutions)


# 📋 تقرير تحليل الأخطاء والحلول

## المشكلة
النموذج يصنف جميع التقييمات كسلبية.

---

## الأسباب المحتملة والحلول

### 1. انقلاب التوسيم (Label Flipping)
**السبب:** تم تبديل التصنيفات بالخطأ
**الحل:**
```python
# عكس التصنيفات
df['label'] = df['label'].map({'positive': 'negative', 'negative': 'positive'})
```

---

### 2. عدم توازن الفئات (Class Imbalance)
**السبب:** فئة أكبر بكثير من الأخرى
**الحلول:**
```python
# الحل 1: استخدام class_weight
model = LogisticRegression(class_weight='balanced')

# الحل 2: Over-sampling للفئة الأقل
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# الحل 3: Under-sampling للفئة الأكبر
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)
```

---

### 3. مشكلة في المعالجة (Preprocessing)
**السبب:** التنظيف أزال كلمات مهمة للتصنيف
**الحلول:**
- مراجعة خطوات التنظيف
- الحفاظ على negation w

In [10]:
# حفظ تقرير التحليل
with open('debug_analysis_report.md', 'w', encoding='utf-8') as f:
    f.write(solutions)

print("✅ تم حفظ التقرير في: debug_analysis_report.md")

✅ تم حفظ التقرير في: debug_analysis_report.md


---

## 📌 ملخص التكليف الرابع

### الأسباب المحتملة:
1. ❌ انقلاب التوسيم → الحل: عكس التصنيفات
2. ❌ عدم توازن الفئات → الحل: class_weight أو SMOTE
3. ❌ مشكلة في المعالجة → الحل: مراجعة خطوات التنظيف
4. ❌ خلل في العتبة → الحل: تعديل threshold

### المخرجات:
- `debug_analysis_report.md` - تقرير التحليل والحلول

---

**✅ انتهى التكليف الرابع**